# Taller Python – Ingeniería Financiera 
**Por: Juan David Calderón - A00403633**

## Librerías necesarias

Antes de empezar, importamos las librerías que vamos a usar a lo largo del taller.
- **pandas**: Manipulación de bases de datos.
- **numpy**: Operaciones matemáticas y vectoriales.
- **yfinance**: Descarga de datos financieros desde Yahoo Finance.
- **ffn**: Funciones financieras útiles (retornos, estadísticas, etc.).
- **plotly**: Visualizaciones interactivas.

Si no estan instalados en consola entonces, si eres windows usar consola normal, si eres usuario mac/Linux debes activar el en entorno virtual y por cada paquete importado tienes que llamar en consola. pip install 'paquete que llames.'

In [72]:
import pandas as pd
import numpy as np
import yfinance as yf
import ffn
import plotly.express as px
import nbformat

---
## 1. Cálculo de Valor Futuro

Vamos a calcular el **Valor Futuro (VF)** usando la fórmula de interés compuesto:

$$VF = VP \times \left(1 + \frac{i_{nom}}{m}\right)^{m \times n}$$

Donde:
* $VP$: Valor Presente
* $i_{nom}$: Tasa nominal anual
* $m$: Número de periodos de capitalización por año (depende de la convención)
* $n$: Número de años

Las convenciones colombianas son:
* **NATV** → Nominal Anual Trimestre Vencido → $m = 4$
* **NAMV** → Nominal Anual Mes Vencido → $m = 12$
* **NABV** → Nominal Anual Bimestre Vencido → $m = 6$
* **NASV** → Nominal Anual Semestre Vencido → $m = 2$

In [73]:
# Diccionario con el número de capitalizaciones por año según la frecuencia
freq_map = {
    'Mensual': 12,
    'Bimestral': 6,
    'Trimestral': 4,
    'Semestral': 2
}

# Datos del Cuadro 1 del taller: (Valor Presente, Tasa Nominal, Años, Frecuencia)
datos_vf = [
    (1_000_000,  0.12, 1, 'Trimestral'),   # 12% NATV
    (5_000_000,  0.18, 2, 'Mensual'),       # 18% NAMV
    (2_500_000,  0.15, 4, 'Bimestral'),     # 15% NABV
    (750_000,    0.20, 3, 'Semestral'),      # 20% NASV
    (12_000_000, 0.10, 2, 'Mensual'),       # 10% NAMV
    (300_000,    0.24, 3, 'Trimestral'),     # 24% NATV
    (8_500_000,  0.14, 2, 'Bimestral'),     # 14% NABV
    (4_200_000,  0.16, 5, 'Semestral'),     # 16% NASV
    (1_500_000,  0.09, 4, 'Mensual'),       # 9% NAMV
    (9_500_000,  0.22, 6, 'Trimestral'),    # 22% NATV
]

# Calculamos el Valor Futuro para cada caso
resultados_vf = []

for vp, i_nom, n, freq in datos_vf:
    m = freq_map[freq]                          # Capitalizaciones por año
    vf = vp * (1 + i_nom / m) ** (m * n)        # Fórmula de interés compuesto
    resultados_vf.append({
        'Valor Presente ($)': f"${vp:,.0f}",
        'Tasa Nominal': f"{i_nom:.0%}",
        'Años': n,
        'Frecuencia': freq,
        'm': m,
        'Valor Futuro ($)': f"${vf:,.2f}"
    })

# Creamos un DataFrame con los resultados
df_vf = pd.DataFrame(resultados_vf)
df_vf.index = range(1, len(df_vf) + 1)
df_vf.index.name = 'Caso'
df_vf

,Valor Presente ($),Tasa Nominal,Años,Frecuencia,m,Valor Futuro ($)
Caso,,,,,,
1,"$1,000,000",12%,1,Trimestral,4,"$1,125,508.81"
2,"$5,000,000",18%,2,Mensual,12,"$7,147,514.06"
3,"$2,500,000",15%,4,Bimestral,6,"$4,521,814.87"
4,"$750,000",20%,3,Semestral,2,"$1,328,670.75"
5,"$12,000,000",10%,2,Mensual,12,"$14,644,691.54"
6,"$300,000",24%,3,Trimestral,4,"$603,658.94"
7,"$8,500,000",14%,2,Bimestral,6,"$11,210,484.30"
8,"$4,200,000",16%,5,Semestral,2,"$9,067,484.99"
9,"$1,500,000",9%,4,Mensual,12,"$2,147,108.00"


---
## 2. Cálculo de Valor Presente

Ahora hacemos el proceso inverso. A partir de un **Valor Futuro** conocido, calculamos el **Valor Presente**:

$$VP = \frac{VF}{\left(1 + \frac{i_{nom}}{m}\right)^{m \times n}}$$

In [74]:
# Datos del Cuadro 2 del taller: (Valor Futuro, Tasa Nominal, Años, Frecuencia)
datos_vp = [
    (2_500_000,  0.14, 2, 'Mensual'),       # 14% NAMV
    (8_000_000,  0.10, 2, 'Trimestral'),     # 10% NATV
    (15_000_000, 0.22, 3, 'Bimestral'),     # 22% NABV
    (450_000,    0.12, 4, 'Semestral'),      # 12% NASV
    (3_200_000,  0.16, 3, 'Mensual'),       # 16% NAMV
    (900_000,    0.20, 4, 'Trimestral'),     # 20% NATV
    (20_000_000, 0.09, 2, 'Bimestral'),     # 9% NABV
    (5_600_000,  0.15, 4, 'Semestral'),     # 15% NASV
    (1_200_000,  0.18, 3, 'Mensual'),       # 18% NAMV
    (7_800_000,  0.11, 2, 'Trimestral'),     # 11% NATV
]

# Calculamos el Valor Presente para cada caso
resultados_vp = []

for vf, i_nom, n, freq in datos_vp:
    m = freq_map[freq]
    vp = vf / (1 + i_nom / m) ** (m * n)    # Despejamos VP de la fórmula
    resultados_vp.append({
        'Valor Futuro ($)': f"${vf:,.0f}",
        'Tasa Nominal': f"{i_nom:.0%}",
        'Años': n,
        'Frecuencia': freq,
        'm': m,
        'Valor Presente ($)': f"${vp:,.2f}"
    })

df_vp = pd.DataFrame(resultados_vp)
df_vp.index = range(1, len(df_vp) + 1)
df_vp.index.name = 'Caso'
df_vp

,Valor Futuro ($),Tasa Nominal,Años,Frecuencia,m,Valor Presente ($)
Caso,,,,,,
1,"$2,500,000",14%,2,Mensual,12,"$1,892,524.16"
2,"$8,000,000",10%,2,Trimestral,4,"$6,565,972.57"
3,"$15,000,000",22%,3,Bimestral,6,"$7,844,889.70"
4,"$450,000",12%,4,Semestral,2,"$282,335.57"
5,"$3,200,000",16%,3,Mensual,12,"$1,986,397.40"
6,"$900,000",20%,4,Trimestral,4,"$412,300.37"
7,"$20,000,000",9%,2,Bimestral,6,"$16,727,748.44"
8,"$5,600,000",15%,4,Semestral,2,"$3,139,932.51"
9,"$1,200,000",18%,3,Mensual,12,"$702,107.68"


---
## 3. Cargar el archivo `datos.csv`

Vamos a cargar la base de datos de precios de ETFs. Es importante que la columna `date` quede como el **índice** del DataFrame y se interprete como fecha.

In [75]:
# Cargamos el archivo CSV indicando que la columna "date" es el índice y que debe parsearse como fecha
df = pd.read_csv(filepath_or_buffer="datos.csv", parse_dates=["date"], index_col="date")

# Verificamos la estructura de la base de datos
df.info()

<class 'pandas.DataFrame'>
DatetimeIndex: 3772 entries, 2010-12-31 to 2025-12-30
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   SPY     3772 non-null   float64
 1   EFA     3772 non-null   float64
 2   IJS     3772 non-null   float64
 3   EEM     3772 non-null   float64
 4   AGG     3772 non-null   float64
dtypes: float64(5)
memory usage: 176.8 KB


In [76]:
# ¿Cuántos registros tiene la base de datos?
print(f"La base de datos tiene {df.shape[0]} registros (filas) y {df.shape[1]} columnas (ETFs).")
print(f"ETFs disponibles: {list(df.columns)}")
df.head()

La base de datos tiene 3772 registros (filas) y 5 columnas (ETFs).
ETFs disponibles: ['SPY', 'EFA', 'IJS', 'EEM', 'AGG']


,SPY,EFA,IJS,EEM,AGG
date,,,,,
2010-12-31,96.198418,37.027023,28.463705,34.364559,70.408226
2011-01-03,97.192902,37.332287,29.057617,34.696377,70.328339
2011-01-04,97.139336,37.217808,28.511217,34.855087,70.348335
2011-01-05,97.644218,37.058807,28.752728,34.768509,70.008797
2011-01-06,97.452972,36.632702,28.582472,34.400635,70.022102


La base de datos tiene 3,772 registros (filas) y 5 columnas correspondientes a los ETFs: SPY, EFA, IJS, EEM y AGG. El rango de fechas va del 31 de diciembre de 2010 al 30 de diciembre de 2025.

---
## 4. Estadísticas descriptivas de cada ETF

Para cada serie de precios calculamos:
* **Promedio** (`.mean()`)
* **Mínimo** (`.min()`)
* **Máximo** (`.max()`)
* **Desviación estándar** (`.std()`)

In [77]:
# a) Promedio
print("Promedio de precios:")
print(df.mean())

Promedio de precios:
SPY    280.836124
EFA     54.177518
IJS     65.292900
EEM     35.858857
AGG     87.132212
dtype: float64


In [78]:
# b) Mínimo
print("Precio mínimo:")
print(df.min())

Precio mínimo:
SPY    85.319344
EFA    30.115509
IJS    22.450733
EEM    22.630241
AGG    69.377312
dtype: float64


In [79]:
# c) Máximo
print("Precio máximo:")
print(df.max())

Precio máximo:
SPY    690.380005
EFA     96.570000
IJS    117.678825
EEM     55.219769
AGG    102.323410
dtype: float64


In [80]:
# d) Desviación estándar
print("Desviación estándar:")
print(df.std())

Desviación estándar:
SPY    153.497118
EFA     14.226447
IJS     24.755653
EEM      6.268967
AGG      8.239981
dtype: float64


In [81]:
# Resumen consolidado en una sola tabla
estadisticas = pd.DataFrame({
    'Promedio': df.mean(),
    'Mínimo': df.min(),
    'Máximo': df.max(),
    'Desv. Estándar': df.std()
}).round(4)

estadisticas

,Promedio,Mínimo,Máximo,Desv. Estándar
SPY,280.8361,85.3193,690.3800,153.4971
EFA,54.1775,30.1155,96.5700,14.2264
IJS,65.2929,22.4507,117.6788,24.7557
EEM,35.8589,22.6302,55.2198,6.2690
AGG,87.1322,69.3773,102.3234,8.2400


---
## 5. Precio de los ETFs el 28 de noviembre de 2025

Usamos `.loc[]` para filtrar por una fecha específica en el índice.

In [82]:
# Precios del 28 de noviembre de 2025
df.loc["2025-11-28"]

SPY    681.376587
EFA     93.495819
IJS    112.831528
EEM     53.553616
AGG    100.159424
Name: 2025-11-28 00:00:00, dtype: float64

---
## 6. Precio del SPY el 30 de diciembre de 2025

Filtramos por fecha y por la columna específica del ETF.

In [83]:
# Precio de SPY el 30 de diciembre de 2025
precio_spy = df.loc["2025-12-30", "SPY"]
print(f"El precio de SPY el 30 de diciembre de 2025 fue: ${precio_spy:.2f}")

El precio de SPY el 30 de diciembre de 2025 fue: $687.01


---
## 7. Descarga de precios de 15 acciones de EE.UU.

Seleccionamos 15 acciones representativas del mercado estadounidense y descargamos los **precios de cierre ajustados** diarios entre el 31 de diciembre de 2022 y el 31 de enero de 2026 usando **yfinance**.

In [84]:
# Definimos la lista de tickers
activos = ["AAPL", "MSFT", "AMZN", "GOOGL", "META",
           "TSLA", "NVDA", "JPM", "V", "JNJ",
           "PG", "UNH", "HD", "KO", "PFE"]

# Descargamos la información de Yahoo Finance
df_acciones = yf.download(activos,
                          start="2022-12-31",
                          end="2026-01-31",
                          progress=False,
                          auto_adjust=False,
                          multi_level_index=False)

df_acciones.head()

Price        Adj Close                                                \
Ticker            AAPL       AMZN      GOOGL          HD         JNJ   
Date                                                                   
2023-01-03  123.096031  85.820000  88.451683  292.711243  162.655853   
2023-01-04  124.365669  85.139999  87.419487  296.250732  164.426743   
2023-01-05  123.046814  83.120003  85.553581  292.303528  163.212708   
2023-01-06  127.574196  86.080002  86.685028  294.212280  164.536301   
2023-01-09  128.095856  87.360001  87.359924  294.471741  160.273422   

Price                                                                 ...  \
Ticker             JPM         KO        META        MSFT       NVDA  ...   
Date                                                                  ...   
2023-01-03  124.928711  57.519154  123.874695  233.452805  14.300684  ...   
2023-01-04  126.093666  57.491737  126.486450  223.240814  14.734249  ...   
2023-01-05  126.065720  56.833858  126.059433  216.624481  14.250736  ...   
2023-01-06  128.478073  57.930328  129.118073  219.177444  14.844139  ...   
2023-01-09  127.947144  57.208481  128.571884  221.311447  15.612371  ...   

Price         Volume                                                     \
Ticker           JPM        KO      META      MSFT       NVDA       PFE   
Date                                                                      
2023-01-03  11054800  12180500  35528500  25740000  401277000  15603800   
2023-01-04  11687600  13387900  32397100  50623400  431324000  21808400   
2023-01-05   8381300   9814700  25447100  39585600  389168000  20057400   
2023-01-06  10029100   9990000  27584500  43613600  405044000  29635900   
2023-01-09   8482300   9442600  26649100  27369800  504231000  30839100   

Price                                             
Ticker           PG       TSLA      UNH        V  
Date                                              
2023-01-03  6447300  231402800  3525600  4202800  
2023-01-04  7313400  180389000  5070400  6606200  
2023-01-05  5373800  157986300  5497100  5246000  
2023-01-06  7882200  220911100  4841300  6829700  
2023-01-09  5727000  190284000  3702800  6294500  

[5 rows x 90 columns]

In [85]:
# Seleccionamos solo los precios de cierre ajustados
df_acciones = df_acciones.loc[:, ["Adj Close"]]

# Eliminamos la primera fila de nombres de variables (multi-index)
df_acciones.columns = df_acciones.columns.droplevel()

# Ordenamos las columnas con el mismo orden de nuestra lista de activos
df_acciones = df_acciones[activos]

# Dimensiones de la base de datos
print(f"Dimensiones: {df_acciones.shape[0]} filas x {df_acciones.shape[1]} columnas")
df_acciones.head()

Dimensiones: 772 filas x 15 columnas


Ticker,AAPL,MSFT,AMZN,GOOGL,META,TSLA,NVDA,JPM,V,JNJ,PG,UNH,HD,KO,PFE
Date,,,,,,,,,,,,,,,
2023-01-03,123.096031,233.452805,85.820000,88.451683,123.874695,108.099998,14.300684,124.928711,202.337952,162.655853,139.523849,490.979919,292.711243,57.519154,42.382496
2023-01-04,124.365669,223.240814,85.139999,87.419487,126.486450,113.639999,14.734249,126.093666,207.430786,164.426743,140.131363,477.593994,296.250732,57.491737,41.448200
2023-01-05,123.046814,216.624481,83.120003,85.553581,126.059433,110.339996,14.250736,126.065720,205.967316,163.212708,138.391602,463.829407,292.303528,56.833858,41.059601
2023-01-06,127.574196,219.177444,86.080002,86.685028,129.118073,113.059998,14.844139,128.478073,212.445572,164.536301,141.687057,463.867310,294.212280,57.930328,42.101387
2023-01-09,128.095856,221.311447,87.360001,87.359924,128.571884,119.769997,15.612371,127.947144,213.274872,160.273422,139.956482,463.924133,294.471741,57.208481,40.009544



La base tiene 772 filas (días de trading entre el 3 de enero de 2023 y el 30 de enero de 2026) y 15 columnas (una por cada acción seleccionada).

---
## 8. Cálculo de retornos a distintas frecuencias

Usamos `resample()` para cambiar la frecuencia de los datos y luego calculamos los retornos simples con el método `.to_returns()` de la librería **ffn**.

Las reglas de resampleo más comunes son:
* `"W"` → Semanal
* `"ME"` → Mensual (fin de mes)
* `"QE"` → Trimestral (fin de trimestre)

### a) Retornos diarios

In [86]:
# Retornos simples diarios
ret_diarios = df_acciones.to_returns()
ret_diarios.dropna(inplace=True)

print(f"Retornos diarios: {ret_diarios.shape[0]} observaciones")
ret_diarios.head()

Retornos diarios: 771 observaciones


Ticker,AAPL,MSFT,AMZN,GOOGL,META,TSLA,NVDA,JPM,V,JNJ,PG,UNH,HD,KO,PFE
Date,,,,,,,,,,,,,,,
2023-01-04,0.010314,-0.043743,-0.007924,-0.011670,0.021084,0.051249,0.030318,0.009325,0.025170,0.010887,0.004354,-0.027264,0.012092,-0.000477,-0.022044
2023-01-05,-0.010605,-0.029638,-0.023726,-0.021344,-0.003376,-0.029039,-0.032816,-0.000222,-0.007055,-0.007383,-0.012415,-0.028821,-0.013324,-0.011443,-0.009376
2023-01-06,0.036794,0.011785,0.035611,0.013225,0.024263,0.024651,0.041640,0.019136,0.031453,0.008110,0.023813,0.000082,0.006530,0.019293,0.025373
2023-01-09,0.004089,0.009736,0.014870,0.007786,-0.004230,0.059349,0.051753,-0.004132,0.003904,-0.025908,-0.012214,0.000122,0.000882,-0.012461,-0.049686
2023-01-10,0.004456,0.007617,0.028732,0.004545,0.027188,-0.007681,0.017981,0.008954,0.011390,-0.002392,-0.000987,-0.008285,0.008873,-0.007666,-0.015913


### b) Retornos semanales

In [87]:
# Re-muestreamos a frecuencia semanal y calculamos retornos
df_semanal = df_acciones.resample(rule="W").ffill()
ret_semanales = df_semanal.to_returns()
ret_semanales.dropna(inplace=True)

print(f"Retornos semanales: {ret_semanales.shape[0]} observaciones")
ret_semanales.head()

Retornos semanales: 160 observaciones


Ticker,AAPL,MSFT,AMZN,GOOGL,META,TSLA,NVDA,JPM,V,JNJ,PG,UNH,HD,KO,PFE
Date,,,,,,,,,,,,,,,
2023-01-15,0.039655,0.063575,0.139870,0.054729,0.053530,0.082611,0.137291,0.036755,0.024386,-0.037837,-0.019750,-0.000878,0.043083,-0.031073,-0.060291
2023-01-22,0.023078,0.004138,-0.008867,0.064047,0.017448,0.090033,0.055624,-0.055451,0.005604,-0.027043,-0.046480,-0.005821,-0.048942,-0.021976,-0.057262
2023-01-29,0.058461,0.033053,0.051311,0.013773,0.088757,0.333383,0.141600,0.038792,0.031786,-0.003022,-0.016787,-0.001377,0.005365,0.006824,-0.020350
2023-02-05,0.058727,0.041062,0.011248,0.054443,0.229274,0.067903,0.036091,0.005487,-0.005660,-0.021518,0.014512,-0.028865,0.046765,-0.010911,0.006166
2023-02-12,-0.021097,0.018386,-0.055905,-0.097442,-0.066370,0.036372,0.007820,-0.000354,-0.010798,-0.014944,-0.030503,0.047096,-0.045128,-0.003510,-0.004086


### c) Retornos mensuales

In [88]:
# Re-muestreamos a frecuencia mensual y calculamos retornos
df_mensual = df_acciones.resample(rule="ME").ffill()
ret_mensuales = df_mensual.to_returns()
ret_mensuales.dropna(inplace=True)

print(f"Retornos mensuales: {ret_mensuales.shape[0]} observaciones")
ret_mensuales.head()

Retornos mensuales: 36 observaciones


Ticker,AAPL,MSFT,AMZN,GOOGL,META,TSLA,NVDA,JPM,V,JNJ,PG,UNH,HD,KO,PFE
Date,,,,,,,,,,,,,,,
2023-02-28,0.023183,0.009018,-0.086299,-0.088830,0.174330,0.187565,0.188309,0.024221,-0.042740,-0.055426,-0.033853,-0.046576,-0.085233,-0.029517,-0.081295
2023-03-31,0.118649,0.155882,0.096148,0.151788,0.211501,0.008507,0.196663,-0.090966,0.025098,0.011353,0.080910,-0.003502,0.002418,0.050341,0.005669
2023-04-30,0.028987,0.065765,0.020912,0.034802,0.133906,-0.207992,-0.001008,0.069180,0.032245,0.056129,0.058303,0.041262,0.018365,0.034177,-0.046814
2023-05-31,0.046059,0.071105,0.143480,0.144681,0.101531,0.241130,0.363436,-0.018301,-0.048421,-0.045633,-0.088758,-0.009856,-0.050074,-0.069992,-0.011792
2023-06-30,0.094330,0.036999,0.081108,-0.025800,0.084089,0.283627,0.118211,0.071697,0.074424,0.067458,0.064842,-0.009496,0.095925,0.017074,-0.035245


### d) Retornos trimestrales

In [89]:
# Re-muestreamos a frecuencia trimestral y calculamos retornos
df_trimestral = df_acciones.resample(rule="QE").ffill()
ret_trimestrales = df_trimestral.to_returns()
ret_trimestrales.dropna(inplace=True)

print(f"Retornos trimestrales: {ret_trimestrales.shape[0]} observaciones")
ret_trimestrales.head()

Retornos trimestrales: 12 observaciones


Ticker,AAPL,MSFT,AMZN,GOOGL,META,TSLA,NVDA,JPM,V,JNJ,PG,UNH,HD,KO,PFE
Date,,,,,,,,,,,,,,,
2023-06-30,0.177916,0.183782,0.262078,0.153957,0.354063,0.261785,0.523072,0.124867,0.055367,0.075928,0.026902,0.021209,0.060166,-0.021786,-0.091253
2023-09-30,-0.116146,-0.070833,-0.024854,0.093233,0.046101,-0.044123,0.028381,0.003960,-0.029634,-0.052194,-0.032694,0.053081,-0.021087,-0.063033,-0.085617
2023-12-31,0.126007,0.193359,0.195249,0.067477,0.179041,-0.006954,0.138563,0.181597,0.134317,0.014415,0.011003,0.047752,0.154610,0.061079,-0.120345
2024-03-31,-0.108197,0.120886,0.187179,0.080464,0.373305,-0.292539,0.824648,0.184804,0.073940,0.016914,0.114199,-0.056641,0.113531,0.046485,-0.021619
2024-06-30,0.229914,0.064262,0.071349,0.208236,0.039418,0.125661,0.367371,0.015676,-0.057776,-0.068579,0.023020,0.033799,-0.096369,0.048445,0.023494


---
## 9. Mejor retorno medio y mayor desviación estándar

Para cada frecuencia, identificamos:
* La acción con el **mejor retorno promedio**.
* La acción con la **desviación estándar más elevada** (mayor riesgo).

In [90]:
# Creamos un diccionario con los retornos de cada frecuencia
frecuencias = {
    'Diario': ret_diarios,
    'Semanal': ret_semanales,
    'Mensual': ret_mensuales,
    'Trimestral': ret_trimestrales
}

resumen = []

for nombre, ret in frecuencias.items():
    mejor_ret = ret.mean().idxmax()
    mejor_ret_val = ret.mean().max()
    mayor_std = ret.std().idxmax()
    mayor_std_val = ret.std().max()
    resumen.append({
        'Frecuencia': nombre,
        'Mejor Retorno Medio': f"{mejor_ret} ({mejor_ret_val:.6f})",
        'Mayor Desv. Estándar': f"{mayor_std} ({mayor_std_val:.6f})"
    })

df_resumen = pd.DataFrame(resumen).set_index('Frecuencia')
df_resumen

,Mejor Retorno Medio,Mayor Desv. Estándar
Frecuencia,,
Diario,NVDA (0.003851),TSLA (0.037553)
Semanal,NVDA (0.018135),TSLA (0.079974)
Mensual,NVDA (0.072103),TSLA (0.159388)
Trimestral,NVDA (0.203456),NVDA (0.286609)


NVDA (Nvidia) es la acción con el mejor retorno promedio en todas las frecuencias: diario (0.39%), semanal (1.81%), mensual (7.21%) y trimestral (20.35%). Esto refleja el rally impulsado por la demanda de GPUs para inteligencia artificial durante 2023–2025.

TSLA (Tesla) presenta la mayor desviación estándar en frecuencia diaria (3.76%), semanal (8.00%) y mensual (15.94%), lo que la posiciona como la acción más volátil del portafolio en el corto plazo. Sin embargo, a frecuencia trimestral es NVDA la de mayor volatilidad (28.66%), lo que indica que sus movimientos de gran magnitud se acumulan en horizontes más largos.

Esto es consistente con el principio financiero de que mayor retorno esperado suele venir acompañado de mayor riesgo.

---
## 10. Matriz de correlaciones de retornos semanales

La matriz de correlaciones nos permite observar qué tan relacionados están los movimientos de las acciones entre sí. Valores cercanos a 1 indican que se mueven juntas, cercanos a -1 que se mueven en dirección opuesta, y cercanos a 0 que tienen poca relación lineal.

In [91]:
# Calculamos la matriz de correlaciones de los retornos semanales
cor_matrix = ret_semanales.corr()

# Visualizamos con Plotly (igual que en el cuaderno de referencia)
fig = px.imshow(cor_matrix,
                text_auto=".2f",
                color_continuous_scale="RdBu_r",
                aspect="auto",
                zmin=-1, zmax=1,
                title="Matriz de Correlaciones – Retornos Semanales")
fig.show()

In [92]:
# Tabla numérica de la matriz de correlaciones
cor_matrix.round(4)

Ticker,AAPL,MSFT,AMZN,GOOGL,META,TSLA,NVDA,JPM,V,JNJ,PG,UNH,HD,KO,PFE
Ticker,,,,,,,,,,,,,,,
AAPL,1.0000,0.3563,0.4020,0.4640,0.3485,0.4106,0.3614,0.2844,0.3680,0.1385,0.1484,-0.0626,0.3009,0.1472,0.2210
MSFT,0.3563,1.0000,0.5895,0.4034,0.5244,0.3850,0.6144,0.2311,0.3616,-0.1351,0.0123,0.0336,0.2683,-0.0661,0.0607
AMZN,0.4020,0.5895,1.0000,0.5058,0.4847,0.4032,0.5263,0.4259,0.4877,-0.1543,0.0350,-0.0065,0.4124,-0.0500,0.0662
GOOGL,0.4640,0.4034,0.5058,1.0000,0.3684,0.3385,0.3314,0.1739,0.2271,0.0147,0.0925,-0.0085,0.2282,0.0016,0.1460
META,0.3485,0.5244,0.4847,0.3684,1.0000,0.2789,0.4055,0.2574,0.3261,-0.1275,0.0098,-0.0569,0.2680,-0.0178,0.0122
TSLA,0.4106,0.3850,0.4032,0.3385,0.2789,1.0000,0.3757,0.2869,0.2105,-0.1222,-0.1177,-0.0563,0.2252,-0.1177,0.0131
NVDA,0.3614,0.6144,0.5263,0.3314,0.4055,0.3757,1.0000,0.2976,0.2502,-0.1819,-0.1188,-0.0861,0.2381,-0.1280,0.0264
JPM,0.2844,0.2311,0.4259,0.1739,0.2574,0.2869,0.2976,1.0000,0.5359,0.1239,0.1308,0.0867,0.4033,0.1030,0.0982
V,0.3680,0.3616,0.4877,0.2271,0.3261,0.2105,0.2502,0.5359,1.0000,0.1841,0.2648,0.1225,0.3620,0.2236,0.1257


### Análisis de la matriz de correlaciones

- Las correlaciones más altas se dan entre acciones tecnológicas: MSFT–NVDA (0.61), MSFT–AMZN (0.59), AMZN–NVDA (0.53) y MSFT–META (0.52), confirmando que comparten exposición sectorial.

- Las acciones defensivas forman su propio bloque: KO–PG (0.49), JNJ–KO (0.48), JNJ–PFE (0.46) y JNJ–PG (0.41).

- Lo más interesante para diversificación son las correlaciones negativas entre tech y defensivas: NVDA–JNJ (−0.18), AMZN–JNJ (−0.15), MSFT–JNJ (−0.14), NVDA–KO (−0.13). Cuando las tech caen, las defensivas tienden a subir, lo que reduce el riesgo total del portafolio.

- El sector financiero (JPM–V: 0.54) se relaciona moderadamente con el consumo cíclico (JPM–HD: 0.40, V–AMZN: 0.49).

- UNH se comporta como un activo casi independiente, con correlaciones prácticamente nulas con el resto (entre −0.09 y +0.13), lo que la hace ideal para reducir el riesgo de la cartera.

En conclusión, la combinación de acciones tech (altos retornos, alta correlación entre sí) con defensivas como JNJ, KO y PG (correlación negativa con tech) y activos independientes como UNH permite construir un portafolio con mejor balance riesgo-retorno.